In [15]:
import json
import pandas as pd
import os

In [32]:
from util_folder.json_extraction_util import read_file


def read_json_files_from_folder(folder_path):
    data = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        a = read_file(file_path)
        data.append(a)
    return data

In [38]:
input_data = read_json_files_from_folder("json_files/json_fine_tuning_input/json2000")
output_data = read_json_files_from_folder("json_files/json_fine_tuning_output/json2000")

# Convert the JSON data to DataFrames
df = pd.DataFrame(input_data, columns=['input'])
df['output'] = output_data
df

,input,output
0,"{""nodes"":[{""viewpoint"":null,""visibility"":""PUBL...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."
1,"{""nodes"":[{""viewpoint"":null,""visibility"":""PUBL...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."
2,"{""nodes"":[{""viewpoint"":null,""visibility"":""PUBL...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."
3,"{""nodes"":[{""viewpoint"":null,""visibility"":""PUBL...","{""nodes"":[{""isDerived"":false,""visibility"":""PUB..."
4,"{""nodes"":[{""viewpoint"":null,""visibility"":""PUBL...","{""nodes"":[{""isStatic"":false,""isDerived"":false,..."
...,...,...
127,"{""nodes"":[{""viewpoint"":null,""visibility"":""PUBL...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."
128,"{""nodes"":[{""viewpoint"":null,""visibility"":""PUBL...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."
129,"{""nodes"":[{""viewpoint"":null,""visibility"":""PUBL...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."
130,"{""nodes"":[{""viewpoint"":null,""visibility"":""PUBL...","{""nodes"":[{""visibility"":""PUBLIC_LITERAL"",""qual..."


In [46]:

def create_dataset(jsonl_file_path, dataset_df):
    with open(jsonl_file_path, 'w') as file:
        for _, row in dataset_df.iterrows():
            # Construct the json object for each line
            json_line = {"input": row['input'], "output": row['output']}
            # Convert the dictionary to a JSON string and write it to the file
            file.write(json.dumps(json_line) + '\n')

    
            

In [48]:
from sklearn.model_selection import train_test_split

def split_dataframe(df, train_size=0.8, validation_size=0.15, test_size=0.05, random_state=None):

    # First split to separate out the training set
    train_df, temp_df = train_test_split(df, train_size=train_size, random_state=random_state)

    # Adjust the proportion sizes for validation and test splits
    proportion = validation_size / (validation_size + test_size)

    # Second split to separate out the validation and test sets
    validation_df, test_df = train_test_split(temp_df, train_size=proportion, random_state=random_state)

    return train_df, validation_df, test_df

train_df, validation_df, test_df = split_dataframe(df)
# Save the datasets
create_dataset("json_files/json_ready/train.jsonl", train_df)
create_dataset("json_files/json_ready/validation.jsonl", validation_df)
create_dataset("json_files/json_ready/test.jsonl", test_df)

In [58]:
import shutil
import os


def zip_folder(folder_path):
    shutil.make_archive(folder_path, 'zip', folder_path)
